
1. Create private AWS ECR repository named "ecs-container-repo"

2. Push the locally stored image to the "ecs-container-repo" repository

3. Create an IAM role named "ecs-task-role" that will be “Elastic Container Service Task" based and have two policies attached to it: “AmazonECSTaskExecutionRolePolicy” and “CloudWatchLogsFullAccess”

4. Create an ECS Cluster named "ecs-demo-cluster" with the launch type as "AWS Fargate (serverless)"

5. Create a task definition under "ecs-demo-cluster", named "ecs-demo-task-definition" with the following configurations:
    - Launch type: AWS Fargate
    - Operating System/Architecture: Linux/X86_64
    - CPU: .5 vCPU
    - Memory: 1 GB
    - Task Role: None
    - Task execution role: ecs-task-role
    - Container name: ecs-demo-app
    - Image URI: <ECR repository URI created in step 2>
    - Container port: 80

6. Create a new security group "ecs-demo-app-sg" with description "ECS Demo App Security Group" and add a inbound rule to allow HTTP traffic on port 80 from anywhere.

7. Deploy task definition

Now run new task "ecs-demo-task-definition" keep “Launch type” as “FARGATE” and the “Platform version” as “LATEST.”
Select us-east-1a as “Subnets” 



Here is a terraform code to create the above resources (not tested): 

```terraform
provider "aws" {
  region = "us-east-1"
}

# 1. Create private AWS ECR repository
resource "aws_ecr_repository" "ecs_container_repo" {
  name = "ecs-container-repo"
}

# 3. Create IAM role for ECS tasks
resource "aws_iam_role" "ecs_task_role" {
  name               = "ecs-task-role"
  assume_role_policy = jsonencode({
    Version = "2012-10-17"
    Statement = [
      {
        Action = "sts:AssumeRole"
        Principal = {
          Service = "ecs-tasks.amazonaws.com"
        }
        Effect = "Allow"
        Sid    = ""
      },
    ]
  })
}

resource "aws_iam_role_policy_attachment" "ecs_task_execution_role_policy" {
  role       = aws_iam_role.ecs_task_role.name
  policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonECSTaskExecutionRolePolicy"
}

resource "aws_iam_role_policy_attachment" "cloudwatch_full_access" {
  role       = aws_iam_role.ecs_task_role.name
  policy_arn = "arn:aws:iam::aws:policy/CloudWatchLogsFullAccess"
}

# 4. Create an ECS Cluster with Fargate
resource "aws_ecs_cluster" "ecs_demo_cluster" {
  name = "ecs-demo-cluster"
}

# 5. Create a task definition
resource "aws_ecs_task_definition" "ecs_demo_task_definition" {
  family                   = "ecs-demo-task-definition"
  requires_compatibilities = ["FARGATE"]
  network_mode             = "awsvpc"
  cpu                      = "512" # .5 vCPU
  memory                   = "1024" # 1 GB
  execution_role_arn       = aws_iam_role.ecs_task_role.arn
  container_definitions    = jsonencode([
    {
      name      = "ecs-demo-app"
      image     = "${aws_ecr_repository.ecs_container_repo.repository_url}:latest"
      cpu       = 512
      memory    = 1024
      essential = true
      portMappings = [
        {
          containerPort = 80
          hostPort      = 80
        }
      ]
    }
  ])
}

# 6. Create a Security Group
resource "aws_security_group" "ecs_demo_app_sg" {
  name        = "ecs-demo-app-sg"
  description = "ECS Demo App Security Group"

  ingress {
    from_port   = 80
    to_port     = 80
    protocol    = "tcp"
    cidr_blocks = ["0.0.0.0/0"]
  }

  egress {
    from_port   = 0
    to_port     = 0
    protocol    = "-1"
    cidr_blocks = ["0.0.0.0/0"]
  }
}
```